In [1]:
import os, json
from ase import Atoms, Atom, io
from ase.io import read, write
from ase.build import bulk, molecule, surface, add_adsorbate
from ase.units import Bohr,Hartree,mol,kcal,kJ,eV
from sparc import SPARC

Successfully registered sparc format with ase.io!


In [2]:
# experimental_gas_formation_energies from Catmap
# The values needs to be subtracted to get the formation energy corresponding to the calculation.
E_expt = json.load(open('E_expt.json', 'r'))

In [7]:
cwd = os.getcwd()
E_dict = {}

mol_list = E_expt.keys()
for mol in mol_list:
    
    if mol not in ['graphite']:
        atoms = molecule(mol)
        atoms.cell = [[10,0,0],[0,10,0],[0,0,10]]
        atoms.pbc = True

    parameters = dict(
                    EXCHANGE_CORRELATION = 'GGA_PBE',
                    D3_FLAG=1,   #Grimme D3 dispersion correction
                    SPIN_TYP=1,   #spin-polarized calculation
                    KPOINT_GRID=[1,1,1],  
                    ECUT=1000/Hartree,   #set ECUT (Hartree) or h (Angstrom)
                    #h = 0.15,
                    TOL_SCF=1e-5,
                    RELAX_FLAG=1,
                    TOL_RELAX = 2.00E-04,  #convergence criteria (maximum force) (Ha/Bohr)
                    PRINT_FORCES=1,
                    PRINT_RELAXOUT=1)
    
    dir_i = f"{cwd}/PBE/{mol}"
    parameters['directory'] = dir_i
    
    if os.path.isdir(dir_i):
        print(f"{dir_i} already exists.")
        continue
    
    if mol == 'H2':
        parameters['SPIN_TYP']=0
    elif mol == 'graphite':
        atoms = read('graphite.cif')
        #parameters['RELAX_FLAG'] = 2
        parameters['KPOINT_GRID'] = [6,6,2]
    
    calc = SPARC(atoms = atoms, **parameters)
    atoms.set_calculator(calc)

    eng = atoms.get_potential_energy()
    E_dict[mol] = eng
    
    with open(f'{dir_i}/energy.txt', 'w') as f:
        f.write(str(eng))
    

/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/CH3OCH3 already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/isobutene already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/CH3CH2OH already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/H2CCO already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/CH3COOH already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/butadiene already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/CH3NO2 already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/CH2NHCH2 already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/C3H8 already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/CH3OH already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/CH2OCH2 already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_energy/PBE/CCH already exists.
/home/hice1/nyu49/Data/VIP/Etc/Gas_formation_e

In [15]:
import re, json

def readFile(path):
    f = open(path, 'r')
    content = f.readlines()
    return content

def read_energy(path):
    energy = float(readFile(path)[0])
    return energy

def get_E_form(name):
    E_elem = {}
    E_elem['H'] = E_dict['H2']/2
    E_elem['O'] = E_dict['O2']/2
    E_elem['C'] = E_dict['graphite'] / len(read('graphite.cif'))
    E_elem['N'] = E_dict['N2']/2
    counts = mol_dict[name].symbols.formula.count()    
    E_form = E_dict[name]
    
    #print(counts, E_form)
    for elem in counts:
        #print(counts[elem], E_elem[elem])
        E_form += -counts[elem] * E_elem[elem]
    return E_form 

E_dict, mol_dict, E_form_dict, t_dict = {}, {}, {}, {}

for mol in mol_list:    
    ### get energy dictionary
    dir_mol = f'PBE/{mol}'
    E_dict[mol] = read_energy(f'{dir_mol}/energy.txt')
    
     ### get Atoms dictionary
    try:
        mol_dict[mol] = molecule(mol)    
    except KeyError:
        pass
    mol_dict['graphite'] = read('graphite.cif')
    
for mol in mol_list:   
     ### get E_form dictionary
    E_form = get_E_form(mol)
    
    if E_form != 0:
        E_form_dict[mol] = E_form
        
         ### get time dictionary
        sparc_out = os.path.join(dir_mol, 'SPARC.out')
        with open(sparc_out, 'r') as f: data = f.read()
        t_SPARC = float(re.search(r'Total walltime\s+:\s+([0-9.]+)', data).group(1)) 
        t_SPARC = float(f"{t_SPARC:.3f}")
        t_dict[mol] = t_SPARC

json.dump(E_form_dict, open('E_form_dict_PBE.json','w'))

In [14]:
E_form_dict['H2O']

-2.4809219060525365